In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc

### personal library

In [ ]:
class Jackknife:
    def __init__( self, len_data, binsize ):
        self.binsize = binsize
        self.nbins = int(len_data/self.binsize)
        self.N = self.binsize * self.nbins
        self.jack_avg = []
        self.est = 0
        self.var_est = 0

    def set( self, func, list_of_data ):
        for i in range( self.nbins ):
            self.jack_avg.append( func( i, self.binsize, list_of_data ) )

    def do_it( self ):
        for i in range( 0, self.nbins ):
            self.est += self.jack_avg[i]
        self.est /= self.nbins

        for i in range( 0, self.nbins ):
            self.var_est += ( self.jack_avg[i] - self.est )**2
        self.var_est /= self.nbins
        self.var_est *= self.nbins -1

    def mean( self ):
        return self.est

    def var( self ):
        return self.var_est

    def err( self ):
        return np.sqrt(self.var_est)

def simple_mean(i, binsize, np_data):
    resmpld = np.delete(np_data, np.s_[i*binsize:(i+1)*binsize])
    return np.mean(resmpld)

def format_print(cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'

def format_print_w_exact(exact, cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'+' exact:'+'{ex:.{width}f}'.format(ex=exact, width=i+2)+' ['+'{num:.{width}f}'.format(num=abs(exact-cen)/err, width=2)+' sigma]'

### related to Fourier coefficients

In [ ]:
def Hankel_nu_k(nu,k):
    return sc.gamma(nu+k+0.5)/(sc.gamma(k+1)*sc.gamma(nu-k+0.5))

In [ ]:
def A_nu_k(nu,k):
    return sc.polygamma(0,nu+k+0.5)-sc.polygamma(0,nu-k+0.5)

In [ ]:
def B_nu_k(nu,k):
    return sc.polygamma(1,nu+k+0.5)-sc.polygamma(1,nu-k+0.5)

In [ ]:
def lambdae_asymp(nu,z,kmax=4): # scaled with exp
    s=0.0
    for k in np.arange(kmax):
        s+=Hankel_nu_k(nu,k)*(-0.5/z)**k
    
    return s/np.sqrt(2.0*np.pi*z)

In [ ]:
def d_dnu_ive_numeric(nu,z,eps=0.00001):
    return (sc.ive(nu+eps,z)-sc.ive(nu-eps,z))/(2.0*eps)

In [ ]:
def d_dnu_lambdae_asymp(nu,z,kmax=4): # scaled with exp
    s=0.0
    for k in np.arange(kmax):
        s+=Hankel_nu_k(nu,k)*A_nu_k(nu,k)*(-0.5/z)**k

    return s/np.sqrt(2.0*np.pi*z)

In [ ]:
def d2_dnu2_ive_numeric(nu,z,eps=0.0001):
    return (sc.ive(nu+eps,z)+sc.ive(nu-eps,z)-2.0*sc.ive(nu,z))/eps**2

In [ ]:
def d2_dnu2_lambdae_asymp(nu,z,kmax=4): # scaled with exp
    s=0.0
    for k in np.arange(kmax):
        s+=Hankel_nu_k(nu,k)*(A_nu_k(nu,k)**2+B_nu_k(nu,k))*(-0.5/z)**k
 
    return s/np.sqrt(2.0*np.pi*z)

### exact solutions

In [ ]:
def Z(beta_,theta_,vlat_,nmax_=3):
    res = 0.0

#     for n in range(-nmax_,nmax_+1):
#         res += sc.iv(n,beta_)**vlat_
    for n in range(-nmax_,nmax_+1):
        res += (np.exp(beta_)*lambdae_asymp(n-theta_/(2.0*np.pi),beta_))**vlat_

    return res

In [ ]:
def avg_plaq(beta_,vlat_,nmax_=3):
    denom=0.0
    numer=0.0

    for n in range(-nmax_,nmax_+1):
        denom += sc.ive(n,beta_)**vlat_
        numer += (sc.ive(n+1,beta_)+sc.ive(n-1,beta_))*sc.ive(n,beta_)**(vlat_-1)

    return 0.5*numer/denom

In [ ]:
def avg_plaq_sq(beta_,vlat_,nmax_=3):
    denom=0.0
    numer=0.0

    for n in range(-nmax_,nmax_+1):
        denom += sc.ive(n,beta_)**vlat_
        numer += (sc.ive(n+2,beta_)+2.0*sc.ive(n,beta_)+sc.ive(n-2,beta_))*sc.ive(n,beta_)**(vlat_-1)

    return 0.25*numer/denom

In [ ]:
def avg_plaq_twopt(beta_,vlat_,nmax_=3):
    denom=0.0
    numer=0.0

    for n in range(-nmax_,nmax_+1):
        denom += sc.ive(n,beta_)**vlat_
        numer += (sc.ive(n+1,beta_)+sc.ive(n-1,beta_))**2 * sc.ive(n,beta_)**(vlat_-2)

    return 0.25*numer/denom

In [ ]:
# def Q(beta_,vlat_,nmax_=3):
#     denom=0.0
#     numer=0.0

#     for n in range(-nmax_,nmax_+1):
#         denom += sc.ive(n,beta_)**vlat_
#         numer += d_dnu_lambda_asymp(n,beta_)*sc.ive(n,beta_)**(vlat_-1)

#     return vlat*numer/denom

In [ ]:
def Qsq(beta_,vlat_,nmax_=3):
    denom=0.0
    numer=0.0

    for n in range(-nmax_,nmax_+1):
        denom += sc.ive(n,beta_)**vlat_
        numer += d2_dnu2_lambdae_asymp(n,beta_)*sc.ive(n,beta_)**(vlat_-1)

    numer *= vlat_
    
    return -numer/denom/(2.0*np.pi)**2

### acceptance

In [ ]:
acc=np.loadtxt("./results/accept_reject.dat")
np.mean(acc)

In [ ]:
dh=np.loadtxt("./results/dH.dat")

In [ ]:
plt.plot(dh)
plt.xlabel("MCS")
plt.ylabel("$dH$")
plt.show()

# numerical parameters

In [ ]:
a=1.0/1.8
beta=1/a**2
print("beta="+str(beta))
lx=16
ly=16
vlat=lx*ly

In [ ]:
nskip=200
nconf=800

### average plaquette

In [ ]:
w0_=[]

for n in range(nskip,nconf):
    dat=np.loadtxt("./results/w0/"+str(n)+".dat")
    w0_.append(dat)
    
w0=np.array(w0_)

In [ ]:
plt.plot(w0)
plt.xlabel("MCS")
plt.ylabel("$\\cos \\kappa_x$")
plt.show()

In [ ]:
binsize=4

In [ ]:
jk = Jackknife(nconf-nskip, binsize)
jk.set(simple_mean, w0)
jk.do_it()
mu_w0=jk.mean()
sig_w0=jk.err()
print(format_print_w_exact(avg_plaq(beta,vlat), mu_w0, sig_w0))

### correlator

In [ ]:
corr_=[]

for n in range(nskip,nconf):
    dat=np.loadtxt("./results/corr/"+str(n)+".dat")
    corr_.append(dat)
    
corr=np.array(corr_)

In [ ]:
plt.plot(corr.T[0])
plt.xlabel("MCS")
plt.ylabel("$\\cos^2 \\kappa_x$")
plt.show()

In [ ]:
binsize=4

In [ ]:
for dx in range(int(lx/2)):
    jk = Jackknife(nconf-nskip, binsize)
    jk.set(simple_mean, corr.T[dx])
    jk.do_it()
    mu=jk.mean()
    sig=jk.err()
    if dx==0:
        print("dx="+str(dx)+": "+format_print_w_exact(avg_plaq_sq(beta,vlat),mu, sig))
    else:
        print("dx="+str(dx)+": "+format_print_w_exact(avg_plaq_twopt(beta,vlat),mu, sig))

### topological charge

In [ ]:
Ql_=[]

for n in range(nskip,nconf):
    dat=np.loadtxt("./results/top_ch/"+str(n)+".dat")
    Ql_.append(dat)
    
Ql=np.array(Ql_)

In [ ]:
plt.hist(Ql,bins=np.arange(-5.1,5,0.2))
plt.xlabel("Q")
plt.show()

In [ ]:
plt.plot(Ql)
plt.xlabel("MCS")
plt.ylabel("$Q$")
plt.show()

In [ ]:
binsize=40

In [ ]:
jk = Jackknife(nconf-nskip, binsize)
jk.set(simple_mean, Ql)
jk.do_it()
Qmu=jk.mean()
Qsig=jk.err()
print(format_print_w_exact(0.0, Qmu, Qsig))

### $Q^2$

In [ ]:
Qlsq=Ql**2

In [ ]:
plt.plot(Qlsq)
plt.xlabel("MCS")
plt.ylabel("$Q^2$")
plt.show()

In [ ]:
binsize=20

In [ ]:
jk = Jackknife(nconf-nskip, binsize)
jk.set(simple_mean, Qlsq)
jk.do_it()
Qmu=jk.mean()
Qsig=jk.err()
print(format_print_w_exact(Qsq(beta,vlat), Qmu, Qsig))